In [1]:
Visit this link to see the interactive topic model visualization http://bit.ly/2t0hMuz
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import os, json
import pyLDAvis.gensim
from bs4 import BeautifulSoup
from random import randint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as p

/home/zakk/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
def get_topic_weight(topic_list):
    """
    Η get_topic_weight() δέχεται ως όρισμα μία λίστα από tuples της μορφής (0, u'0.071*"camera" +...+ 0.019*"pictur"') 
    όπως ακριβώς παράγεται από την συνάρτηση print_topics() της gensim. Στο παραπάνω παράδειγμα το 0 #αντιπροσωπεύει
    το topic και μετά ακολουθούν οι "κυρίαρχες" λέξεις ανα topic με το αντίστοιχο "βάρος"/πιθανότητα τους.
    Επιστρέφει τις ίδιες πληροφορίες (id του topic και λέξεις με βάρη) σε επεξεργάσιμο data type.
    Η weight_list που θα προκύψει θα είναι μία λίστα με στοιχεία τόσα όσα και τα topics και έχουν την 
    μορφή: (0, {'summer': 0.025, 'distanc': 0.021, ... , 'shot': 0.005, 'help': 0.001})
    """
    #Αρχικοκοποίηση της λίστας
    weight_list=[]
    for item in topic_list:
        topic = item[1].encode('utf-8')
        #Dictionary στο οποίο θα εισαχθούν οι λέξεις με το αντίστοιχο βάρος 
        w_dict = {}
        while len(topic) > 0:
            start = 0
            star = topic[start:].find('*')
            #Εντοπίζω το βάρος της λέξης
            w = topic[:star]
            prob = float(w)
            #Θέτω νέα αφετηρία για το keyword
            start = star + 2
            qt = topic[start:].find('"') + start
            #Εντοπίζω τη λέξη
            keyword = topic[start:qt]
            #Ανανεώνεται το περιεχόμενο του topic
            topic = topic[qt + 3:]
            #Καταχώρηση του αποτελέσματος στο dictionary
            w_dict[keyword] = prob
            #Καταχώρηση του dictionary με τα βάρη καθώς και του αριθμού του topic στην λίστα 
            #weight_list που θα επιστραφεί από την συνάρτηση
        weight_list.append((item[0], w_dict))
    return weight_list


def score(texts_eval, weight_list):
    """
    Η score() δέχεται δύο ορίσματα. Το πρώτο όρισμα είναι μία λίστα από αντικείμενα όπως το εξής: 
    [[u'love', u'phone', u'much',...., u'perfect'], 0] το οπόιο αποτελείται από δύο κομμάτια.
    Το πρώτο είναι το review προς αξιολόγηση σε μορφή λίστας λέξεων, bow (bag of words) και το δεύτερο είναι
    ο αριθμός του topic από το οποίο προέρχεται (πληροφορία που θα μας χρειαστεί κατά την αξιολόγηση).
    Το δεύτερο όρισμα είναι η λίστα που επιστρέφει η συνάρτηση get_topic_weight().
    Επιστρέφει λίστα που περιέχει τα δεδομένα των reviews όπως είναι στο πρώτο όρισμα, εμπλουτισμένα με το "σκορ" 
    που πέτυχε το εκαστοτε review σε κάθε topic.
    
    Η temp_list που θα προκύψει θα είναι μία λίστα με στοιχεία τόσα όσα και τα topics και έχουν την παρακάτω μορφή: 
    [[u'love', u'phone', u'much',...., u'perfect'], 0, [(0, 0.189), (1, 0.01), ..., (5, 0.086)]]
    Το 1ο κομμάτι ([u'love', u'phone', u'much',...., u'perfect']) ,όπως είπαμε και παραπάνω είναι το review σε bow,
    το 2ο κομμάτι (0) είναι το topic από το οποίο προέρχεται πραγματικά και
    το 3ο κομμάτι ([(0, 0.189), (1, 0.01), ..., (5, 0.086)]) είναι το σκορ που πέτυχε σε κάθε ένα απο τα topics
    """
    temp_list = []
    for yh in texts_eval:
        temp_list.append([yh[0],yh[1]])
    #Πλήθος των topics
    weight_list_len = len(weight_list)
    #Για κάθε review
    for item in temp_list:
        result = []
        # Για κάθε ένα topic
        for k in range(weight_list_len):
            sum = 0
            # Για κάθε dict με βάρη λέξεων
            for i in weight_list[k][1].keys():
                #Πολλαπλασίασε το πλήθος των φορών που βρέθηκε η λέξη i στην λίστα item[0] (review σε bow)
                #με το βάθος της λέξης  weight_list[k][1][i] και πρόσθεσε το αποτέλεσμα στην sum
                sum += item[0].count(i) * weight_list[k][1][i]
            #Εισαγωγή στην λίστα με τα αποτελέσματα του κάθε topic 
            result.append((k,round(sum,3)))
        #Η αρχική λίστα με τα reviews ενημερώνεται με τα scores που πέτυχε στο εκάστοτε topic
        item.append(result)
    return temp_list

def get_eval(subj_topics, texts_eval, ignore_list):
    """
    Η get_eval() δέχεται τρία ορίσματα. Το πρώτο είναι ένα dictionary στο οποίο έχει γίνει μια
    υποκειμενική αντιστοίχιση των topics id's που παράχθηκαν από την LDA με τα αρχικά topics του dataset.
    Το δεύτερο όρισμα είναι η λίστα που επιστρέφει η συνάρτηση score().
    Το τρίτο είναι μία λίστα με τα topics id's τα οποία δεν λαμβάνονται υπόψη κατά την αξιολόγηση, κατά συνέπεια τα 
    reviews που ανήκουν σε αυτά τα topics αφαιρούνται απο το αρχικό dataset. 
    Η λίστα προκύπτει μετά από υποκειμενική εξέταση. 
    Επιστρέφει ποσοστό εύστοχων προβλέψεων των reviews, πλήθος πετυχυμένων προβλέψεων, την νέα μειωμένη τιμή 
    του πλήθους των προς αξιολόγηση reviews και το αρχικό πλήθος αυτών (πριν αφαιρεθούν τα reviews που ανηκουν 
    στην ignore_list)
    """
    #Αρχικοποίηση για το μέγιστο ποσοστό
    maxperc = 0
    final_perc = 0
    #Πλήθος των προς αξιολόγηση reviews
    le = len(texts_eval)
    count = 0
    #Για κάθε review
    for item in texts_eval:
        #Βρες σε ποιο LDA topic επιτέθχθηκε η μέγιστη βαθμολογία
        templ = []
        for p in item[2]:
            templ.append(p[1])
        themax = max(templ)
        themax_index = templ.index(themax)
        new_templ = [n for n in templ if n!=themax]
        if new_templ:
            themax2 = max(n for n in templ if n!=themax)
        else:
            continue
        themax_index2 = templ.index(themax2)
        #Εάν το topic με την μέγιστη βαθμολογία ανήκει στην ignore_list, 
        #τότε μειώνεται κατά ένα το πλήθος των προς αξιολόγηση reviews
        #και η ροή περνάει στην επόμενη επανάληψη (επόμενο review)
        if themax_index in ignore_list or themax_index2 in ignore_list:
            le -= 1
            continue
        #Εάν το topic με την μέγιστη βαθμολογία είναι ίδιο με αυτό που πραγματικά ανήκει το review
        #τότε αυξάνεται κατά ένα το count το οποίο μετράει τις πετυχημένες προβλέψεις
        if item[1] in [subj_topics[themax_index], subj_topics[themax_index2]]:
            count += 1
    #Εντοπισμός μέγιστης τιμής
    if maxperc < count:
        maxperc = count
        #Υπολογισμός ποσοστού
        final_perc = round(maxperc/float(le)*100,3)
    return (final_perc, maxperc, le, len(texts_eval))

def get_eval0(subj_topics, texts_eval, ignore_list):
    #Αρχικοποίηση για το μέγιστο ποσοστό
    maxperc = 0
    final_perc = 0
    #Πλήθος των προς αξιολόγηση reviews
    le = len(texts_eval)
    count = 0
    #Για κάθε review
    for item in texts_eval:
        #Βρες σε ποιο topic επιτέθχθηκε η μέγιστη βαθμολογία
        templ = []
        for p in item[2]:
            templ.append(p[1])
        themax = max(templ)
        themax_index = templ.index(themax)
        #Εάν το topic με την μέγιστη βαθμολογία ανήκει στην ignore_list, 
        #τότε μειώνεται κατά ένα το πλήθος των προς αξιολόγηση reviews
        #και η ροή περνάει στην επόμενη επανάληψη (επόμενο review)
        if themax_index in ignore_list:
            le -= 1
            continue
        #Εάν το topic με την μέγιστη βαθμολογία είναι ίδιο με αυτό που πραγματικά ανήκει το review
        #τότε αυξάνεται κατά ένα το count το οποίο μετράει τις πετυχημένες προβλέψεις
        if item[1] == subj_topics[themax_index]:
            count += 1
    #Εντοπισμός μέγιστης τιμής
    if maxperc < count:
        maxperc = count
        #Υπολογισμός ποσοστού
        final_perc = round(maxperc/float(le)*100,3)
    return (final_perc, maxperc, le, len(texts_eval))

In [3]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_set=[]

dict_topics = {}

In [4]:
# Δημιουργεί μία λίστα με τα ονόματα των φακέλων που βρίσκονται μέσα στο directory data/
folder_names = os.listdir("data/")

# Για κάθε φάκελο
for f in folder_names:
    # Φτιάξε μία λίστα με τα ονόματα των αρχείων που βρίσκονται μέσα στον φάκελο f
    file_names = os.listdir("data/" + f)
    # Αποθήκευσε το όνομα της κατηγορίας και το id αυτής
    dict_topics[folder_names.index(f)] = f
    sum_of_reviews = 0
    # Για κάθε αρχείο
    for it in file_names:
        # Άνοιξε το
        with open("data/" + f + '/' + it) as data_file:  
            data = json.load(data_file)
        # Ενημέρωση αθροίσματος του πλήθους των Reviews
        sum_of_reviews += len(data['Reviews'])
        # Για κάθε Review 
        for i in data['Reviews']:
            try:
                # Αφαίρεσε όλα τα html tags
                soup = BeautifulSoup(i['Content'], "lxml")
                #Ενημέρωση του doc_set και για το topic απο το οποίο προέρχεται το review 
                doc_set.append((soup.get_text(),folder_names.index(f)))
            except:
                pass
    print sum_of_reviews, ' reviews in ', f    
print 'There are ',len(doc_set), ' reviews in the dataset!'
print 'Categories: ', dict_topics
# list for tokenized documents in loop
texts = []
texts_eval = []

4814  reviews in  mobilephone
4578  reviews in  cameras
4677  reviews in  video_surveillance


/home/zakk/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


4461  reviews in  TVs
4484  reviews in  tablets
4320  reviews in  laptops
There are  27320  reviews in the dataset!
Categories:  {0: 'mobilephone', 1: 'cameras', 2: 'video_surveillance', 3: 'TVs', 4: 'tablets', 5: 'laptops'}


In [5]:
# Βρόγχος που διαπερνάει την λίστα των reviews
for item in doc_set:
    try:
        # κατακερματισμός σε λέξεις και αφαίερεση κεφαλαίων γραμματων
        raw = item[0].lower()
        tokens = tokenizer.tokenize(raw)
        # Αφαίρεση των stop words
        stopped_tokens0 = [i for i in tokens if not i in en_stop]
        # Αφαίρεση λέξεων με μήκος < 3
        stopped_tokens = [i for i in stopped_tokens0 if len(i) >= 4]
        # Εφαρμογή αλγορίθμου Porter Stemming
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # Καταχώρηση του επεξεργασμένου κειμένου
        texts.append(stemmed_tokens)
        texts_eval.append([stemmed_tokens,item[1]])        
    except:
        pass
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [14]:
# Πλήθος λέξεων που θα περιέχει το κάθε topic
num_of_words = 250
# Λίστες που θα κρατάνε τα ενδιάμεσα αποτελέσματα του βρόγχου
model_topics_list = []
model_topics = []
review_score = []
n = 6
for num_t in range(n/2, n*2 + 1):
    print num_t
    #Δημιουργία των topics σε αντικείμενο κλάσης από την LDA 
    temp_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=int(num_t), id2word = dictionary, passes=20)
    model_topics.append(temp_model)
    with open('lda_model_'+ str(num_t) +'.pkl', 'wb') as fout:
        p.dump(temp_model, fout)
    print '\ttemp_model OK'
    #Εξαγωγή των topics
    temp_model_topics = temp_model.print_topics(num_topics=int(num_t), num_words=num_of_words)
    with open("OutputNew.txt", "a") as text_file:
        text_file.write("\t\tres%s: %s\n\n\n\n" % (num_t, temp_model_topics))
    print '\ttemp_model_topics OK'
    #Εξαγωγή των βαρών τβων λέξεων ανά topic σε dictionary
    temp_topic_weight = get_topic_weight(temp_model_topics)
    print '\ttemp_topic_weight OK'
    #Ενημέρωση της αρχικής λίστας με την βαθμολογία κάθε review ανά topic
    temp_review_score = score(texts_eval, temp_topic_weight)
    review_score.append(temp_review_score)
    print '\ttemp_review_score OK'

3
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
4
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
5
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
6
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
7
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
8
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
9
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
10
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
11
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK
	temp_review_score OK
12
	temp_model OK
	temp_model_topics OK
	temp_topic_weight OK


/home/zakk/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:68: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


	temp_review_score OK


In [17]:
# Σάρωση όλων των πιθανών map μεταξύ LDA topic και πραγματικού για να δω που προκύπτει μέγιστη απόδοση
# 
lis_subj_rel = []
for tt in range(10):
    print 'topic', tt + 3
    theD = {}
    s = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    for ga in range(tt + 3):
        mmaaxx = 0
        for gb in range(6):
            d = s[:]
            d.remove(ga)
            di = {ga: gb}
            rrr = get_eval0(di, review_score[tt], d)
            if rrr[1] > mmaaxx:
                mmaaxx = rrr[1]
                percc = (rrr[1], rrr[2], round(rrr[2]/float(rrr[3]) * 100, 3))
                mmaaxxa = ga
                mmaaxxb = gb
                theD[ga] = gb
        print percc, {mmaaxxa: mmaaxxb}, rrr
    print ' - ', theD, ' - ', get_eval(theD, review_score[tt], [])
    lis_subj_rel.append(theD)
    print

topic 3
(4348, 12013, 43.971) {0: 1} (4.245, 510, 12013, 27320)
(3411, 9431, 34.52) {1: 3} (11.102, 1047, 9431, 27320)
(2837, 5876, 21.508) {2: 4} (46.886, 2755, 5876, 27320)
 -  {0: 1, 1: 3, 2: 4}  -  (46.684, 12754, 27320, 27320)

topic 4
(3761, 5698, 20.857) {0: 1} (0.404, 23, 5698, 27320)
(1570, 2879, 10.538) {1: 4} (40.465, 1165, 2879, 27320)
(4287, 16390, 59.993) {2: 0} (17.785, 2915, 16390, 27320)
(1412, 2353, 8.613) {3: 2} (8.882, 209, 2353, 27320)
 -  {0: 1, 1: 4, 2: 0, 3: 2}  -  (58.715, 16041, 27320, 27320)

topic 5
(1895, 2611, 9.557) {0: 4} (19.877, 519, 2611, 27320)
(1848, 1995, 7.302) {1: 2} (1.153, 23, 1995, 27320)
(4171, 16037, 58.701) {2: 3} (21.656, 3473, 16037, 27320)
(3557, 4831, 17.683) {3: 1} (0.207, 10, 4831, 27320)
(715, 1846, 6.757) {4: 0} (15.547, 287, 1846, 27320)
 -  {0: 4, 1: 2, 2: 3, 3: 1, 4: 0}  -  (67.943, 18562, 27320, 27320)

topic 6
(3427, 4695, 17.185) {0: 1} (0.319, 15, 4695, 27320)
(2669, 3037, 11.116) {1: 4} (6.684, 203, 3037, 27320)
(270, 889, 3

In [24]:
print '3 topics'
top3a = get_eval(lis_subj_rel[0], review_score[0], [])
print top3a[0],'% accuracy, ', 100 - round(top3a[2]/float(top3a[3])*100,2),'% loss'

3 topics
46.684 % accuracy,  0.0 % loss


In [25]:
print '4 topics'
top4a = get_eval(lis_subj_rel[1], review_score[1], [])
print top4a[0],'% accuracy, ', 100 - round(top4a[2]/float(top4a[3])*100,2),'% loss'

4 topics
58.715 % accuracy,  0.0 % loss


In [26]:
print '5 topics'
top5a = get_eval(lis_subj_rel[2], review_score[2], [])
print top5a[0],'% accuracy, ', 100 - round(top5a[2]/float(top5a[3])*100,2),'% loss'

5 topics
67.943 % accuracy,  0.0 % loss


In [27]:
print '6 topics'
top6a = get_eval(lis_subj_rel[3], review_score[3], [])
print top6a[0],'% accuracy, ', 100 - round(top6a[2]/float(top6a[3])*100,2),'% loss'

6 topics
72.28 % accuracy,  0.0 % loss


In [28]:
print '7 topics'
top7a = get_eval(lis_subj_rel[4],review_score[4] ,[])
print top7a[0],'% accuracy, ', 100 - round(top7a[2]/float(top7a[3])*100,2),'% loss'

7 topics
68.594 % accuracy,  0.0 % loss


In [29]:
print '8 topics'
top8a = get_eval(lis_subj_rel[5],review_score[5] ,[])
print top8a[0],'% accuracy, ', 100 - round(top8a[2]/float(top8a[3])*100,2),'% loss'

8 topics
66.241 % accuracy,  0.0 % loss


In [30]:
print '9 topics'
top9a = get_eval(lis_subj_rel[6],review_score[6] ,[])
print top9a[0],'% accuracy, ', 100 - round(top9a[2]/float(top9a[3])*100,2),'% loss'

9 topics
77.789 % accuracy,  0.0 % loss


In [31]:
print '10 topics'
top10a = get_eval(lis_subj_rel[7],review_score[7] ,[])
print top10a[0],'% accuracy, ', 100 - round(top10a[2]/float(top10a[3])*100,2),'% loss'

10 topics
75.659 % accuracy,  0.0 % loss


In [32]:
print '11 topics'
top11a = get_eval(lis_subj_rel[8],review_score[8] ,[])
print top11a[0],'% accuracy, ', 100 - round(top11a[2]/float(top11a[3])*100,2),'% loss'

11 topics
77.764 % accuracy,  0.0 % loss


In [33]:
print '12 topics'
top12a = get_eval(lis_subj_rel[9], review_score[9] ,[])
print top12a[0],'% accuracy, ', 100 - round(top12a[2]/float(top12a[3])*100,2),'% loss'

12 topics
69.758 % accuracy,  0.0 % loss


In [34]:
# Υποκειμενική αντιστοίχιση για n topics με n = 3, 4, 5, ...., 12
lis_subj_rel

[{0: 1, 1: 3, 2: 4},
 {0: 1, 1: 4, 2: 0, 3: 2},
 {0: 4, 1: 2, 2: 3, 3: 1, 4: 0},
 {0: 1, 1: 4, 2: 4, 3: 3, 4: 0, 5: 5},
 {0: 3, 1: 2, 2: 5, 3: 1, 4: 5, 5: 0, 6: 4},
 {0: 1, 1: 2, 2: 5, 3: 5, 4: 0, 5: 5, 6: 5, 7: 4},
 {0: 1, 1: 3, 2: 4, 3: 5, 4: 0, 5: 1, 6: 5, 7: 5, 8: 2},
 {0: 5, 1: 4, 2: 2, 3: 1, 4: 0, 5: 3, 6: 1, 7: 5, 8: 4, 9: 4},
 {0: 0, 1: 1, 2: 2, 3: 4, 4: 1, 5: 2, 6: 5, 7: 3, 8: 0, 9: 5, 10: 5},
 {0: 1, 1: 5, 2: 5, 3: 4, 4: 5, 5: 4, 6: 1, 7: 0, 8: 5, 9: 2, 10: 4, 11: 4}]

In [40]:
lda_model_list = []
for mdl in range(10):
    lda_model_list.append(p.load(open('lda_model_'+ str(mdl+3) +'.pkl')))

# Intertopic Distance Map for 3 topics (via multidimensional scaling)

In [42]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[0], corpus, dictionary))

# Intertopic Distance Map for 4 topics (via multidimensional scaling)

In [43]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[1], corpus, dictionary))

# Intertopic Distance Map for 5 topics (via multidimensional scaling)

In [46]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[2], corpus, dictionary))

# Intertopic Distance Map for 6 topics (via multidimensional scaling)

In [47]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[3], corpus, dictionary))

# Intertopic Distance Map for 7 topics (via multidimensional scaling)

In [48]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[4], corpus, dictionary))

# Intertopic Distance Map for 8 topics (via multidimensional scaling)

In [49]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[5], corpus, dictionary))

# Intertopic Distance Map for 9 topics (via multidimensional scaling)

In [50]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[6], corpus, dictionary))

# Intertopic Distance Map for 10 topics (via multidimensional scaling)

In [51]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[7], corpus, dictionary))

# Intertopic Distance Map for 11 topics (via multidimensional scaling)

In [52]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[8], corpus, dictionary))

# Intertopic Distance Map for 12 topics (via multidimensional scaling)

In [53]:
pyLDAvis.display(data=pyLDAvis.gensim.prepare(lda_model_list[9], corpus, dictionary))